In [1]:
import pandas as pd
import numpy as np

In [2]:
# Διαβάζω τα csv
df_icu_stay = pd.read_csv('D:\MSc Aegean\Thesis\mimic-iv-1.0\icu\icustays.csv.gz', usecols = ['subject_id', 'hadm_id', 'los'])
df_hours = pd.read_csv('D:\Papers\Cyprus_Paper\Data\Twenty_four_hours\o3 hours\o1_3hours.csv')
df_hours = df_hours.drop(['Survival Days'], axis=1)

In [3]:
'''
Ομαδοποίηση ημερών παραμονής στη ΜΕΘ
μιας και υπάρχουν πολλαπλές εγγραφές
λόγο μετακινήσεων με το ίδιο hadm_id.
Έτσι παίρνω το άθροισμα των ημερών
παραμονής.
'''
df_temp = df_icu_stay.groupby(df_icu_stay['hadm_id']).aggregate({'subject_id': 'first', 'hadm_id': 'first', 'los': 'sum'}).reset_index(drop=True)

In [4]:
# Ελέγχω αν μετά την συγχώνευση υπάρχουν διπλές ίδιες στήλες

rslt_df=pd.merge(df_hours, df_temp, on=['subject_id','hadm_id'],
              how="left", suffixes=('', '_delme'))

# Και αν υπάρχουν τις αφαιρώ
rslt_df = rslt_df[[c for c in rslt_df.columns if not c.endswith('_delme')]]

In [5]:
# Αντικαθιστώ τις τιμές στην στήλη Los στους επιζήσαντες με το 365
rslt_df.loc[rslt_df.hospital_expire_flag == 'survival', 'los'] = 365

# Μετενομ΄άζω την στήλη los σε Survival Days
rslt_df = rslt_df.rename(columns={"los": "Survival Days"})

In [9]:
rslt_df

,subject_id,hadm_id,date,time,Mean - Heart Rate (bpm),Mean - Respiratory Rate (insp/min),Mean - O2 saturation pulseoxymetry (%),Mean - Non Invasive Blood Pressure systolic (mmHg),Mean - Non Invasive Blood Pressure diastolic (mmHg),Mean - Non Invasive Blood Pressure mean (mmHg),...,"Std Dev - Glucose, Urine (Urine)",Std Dev - Myelocytes (Cerebrospinal Fluid),Std Dev - Mesothelial Cells (Joint Fluid),Std Dev - NRBC (Joint Fluid),Std Dev - I (Blood),hospital_expire_flag,gender,ethnicity,Age,Survival Days
0,10004733,27411876,2174-12-04,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,M,UNKNOWN,51,365.000000
1,10006277,25610553,2176-06-07,21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,M,WHITE,88,365.000000
2,10008100,29402054,2181-12-10,18:00:00,91.0,16.0,99.0,180.0,64.0,95.0,...,NaN,NaN,NaN,NaN,NaN,0,F,WHITE,86,365.000000
3,10017492,27417763,2116-06-23,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,M,OTHER,86,1.178299
4,10025463,24470193,2137-10-07,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,M,WHITE,67,0.611944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3641,19992885,20352341,2152-07-16,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,M,WHITE,50,7.774468
3642,19994233,29338696,2184-02-07,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,F,UNKNOWN,87,365.000000
3643,19997293,26366652,2123-11-14,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,M,WHITE,76,365.000000
3644,19999442,26785317,2148-11-18,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,M,WHITE,43,365.000000


In [8]:
# αντικατάσταση του survival και death με 0 & 1
rslt_df["hospital_expire_flag"].replace({"survival": 0, "death": 1}, inplace=True)

In [10]:
rslt_df.to_csv('D:\Papers\Cyprus_Paper\Data\Twenty_four_hours\o3 hours\o1_3hours.csv', index=False)